In [2]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import re
from IPython.core.display import HTML

# Utilized https://github.com/wjakethompson/taylor

In [3]:
# songs_subset favors Taylor's Version over original, exludes remixes, demos, acoustic, and pop versions of songs. This eliminates duplication of lyrics in analysis.
# Removed songs placed in songs_subset_removed

# Load the aggregate data
aggregate_df = pd.read_csv('taylor/data/taylor_song_sound_data.csv')


songs_subset_path = 'taylor/data/songs_subset'

# Function to standardize song titles and filenames
def standardize_name(name):
    # Convert to lowercase
    name = name.lower()
    # Remove special characters and common substrings
    name = re.sub(r'[^a-z0-9]', '', name)
    name = re.sub(r'popversion', '', name)
    return name


# Loop through each file in the songs_subset folder
for filename in os.listdir(songs_subset_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(songs_subset_path, filename)
        
        # Check if the file exists before attempting to open it
        if os.path.isfile(file_path):
            # Read the individual song file
            song_df = pd.read_csv(file_path)
            lyrics = ' '.join(song_df['lyric'].tolist())
            
            # Attempt to match the song title in aggregate_df to the filename
            # The matching logic will depend on the actual column name and your data
            standardized_filename = standardize_name(filename.replace('.csv', ''))
            
            # Find the corresponding track name in the aggregate dataframe
            for song_title in aggregate_df['track_name']:
                standardized_song_title = standardize_name(song_title)
                # Check for a match using the standardized names
                if standardized_song_title == standardized_filename:
                    # If a match is found, update the lyrics column
                    idx = aggregate_df.index[aggregate_df['track_name'] == song_title].tolist()[0]
                    aggregate_df.at[idx, 'lyrics'] = lyrics
                    break 
        else:
            print(f"File not found: {file_path}")

# Save the updated DataFrame
aggregate_df.to_csv('taylor_song_sound_data_updated.csv', index=False)

C:\Users\nmack\AppData\Local\Temp\ipykernel_17776\1126452840.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'It feels like a perfect night To dress up like hipsters And make fun of our exes Uh-uh, uh-uh It feels like a perfect night For breakfast at midnight To fall in love with strangers Uh-uh, uh-uh Yeah We're happy, free, confused, and lonely at the same time It's miserable and magical, oh yeah Tonight's the night when we forget about the deadlines It's time, oh-oh I don't know about you But I'm feelin' twenty-two Everything will be alright if You keep me next to you You don't know about me But I'll bet you want to Everything will be alright if We just keep dancin' like we're Twenty-two Twenty-two It seems like one of those nights This place is too crowded Too many cool kids Uh-uh, uh-uh (Who's Taylor Swift anyway? Ew) It seems like one of those nights We ditch the whole scene And end up dreamin' Instead of

In [4]:
# This will return a Series of booleans indicating whether each row contains the phrase
file_not_found = aggregate_df['lyrics'].str.contains('File not found', na=False)

# To simply check if any row contains "file not found" in 'lyrics':
any_file_not_found = file_not_found.any()

print(any_file_not_found)

False


## Descriptive Analysis
- How varied is the vocabulary used across different albums?
- What words and/or phrases are repeated in each album and throughout all albums?

In [6]:
import spacy

In [7]:
nlp = spacy.load('en_core_web_sm')